In [6]:
# Install LangChain, LiteLLM, FAISS, and other dependencies
!pip install langchain langchain_community faiss-cpu pypdf tiktoken litellm sentence-transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.5 MB/s eta 0:00:00


In [8]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the uploaded PDF
pdf_path = "/content/testPDF.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Split text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

print(f"Total chunks created: {len(chunks)}")

Total chunks created: 3


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Load the SentenceTransformer model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

# Create FAISS vectorstore from document chunks
vectorstore = FAISS.from_documents(chunks, embedding_model)

# Convert to retriever
# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retriever = vectorstore.as_retriever(
    search_type="mmr",  # Change search_type to "mmr"
    search_kwargs={"k": 3, "lambda_mult": 0.3} # Add lambda_mult here
)

In [12]:
import litellm
from google.colab import userdata
# Set your GROQ API key here
litellm.api_key = userdata.get('GROQ_API_KEY')

# Define a function to call Groq's llama3-70b-8192 using LiteLLM
def generate_answer_with_litellm(model_name, context, question):
    response = litellm.completion(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions based on the company's employee handbook."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {question}"}
        ]
    )
    return response['choices'][0]['message']['content']

In [13]:
# Function that retrieves context and generates answer using LLM
def rag_qa(question, model_name):
    # Step 1: Retrieve relevant context
    relevant_docs = retriever.get_relevant_documents(question)
    combined_context = "\n".join([doc.page_content for doc in relevant_docs])

    # Step 2: Generate answer
    answer = generate_answer_with_litellm(model_name, combined_context, question)
    return answer, combined_context

In [14]:
# LLaMA3 model name for Groq via LiteLLM
llama3_model_name = "groq/llama-3.3-70b-versatile"  # This is the LiteLLM model ID for LLaMA3-70B via Groq

# Your test question
user_question = "What is the GIT command for changing branch?"

# Generate answer
try:
    answer, context = rag_qa(user_question, llama3_model_name)
    print("="*80)
    print(f"Answer from {llama3_model_name}:\n\n{answer}")
except Exception as e:
    print(f"Error: {e}")

/tmp/ipython-input-13-656141505.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(question)


Answer from groq/llama-3.3-70b-versatile:

According to the company's employee handbook, the GIT command to change a branch is not explicitly mentioned in the provided text. However, in general Git terminology, the command to switch or change to a different branch is:

`git checkout <branch-name>`

or, alternatively, using the newer switch command:

`git switch <branch-name>`

Please note that the exact command may vary depending on the specific Git version or configuration used within the company. For the most accurate and up-to-date information, I recommend consulting the company's Git documentation or seeking guidance from the IT department.
